# Find the Top Ten Words for Story WV, Loc Template WV, and Soc Template WV
## For each story there will be 3 'Top Ten Words', One for each type of regressor
## For each regressor, regress out of the recall wv the other two vectors. Do this for each participant.
## Correlate the residual wvs with each word from the target regressor. 
## Sum correlation values across participants
## Index the 10 highest values, and find the 10 highest words
## Put in dictionary


In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from scipy import stats


from sklearn.decomposition import PCA #for cluster analysis
from gensim.models import KeyedVectors #for word embeddings
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

# from _DRAFT_20200604_functions import * #includes constants and score function
from tqdm import tqdm_notebook #for progress bar

%autosave 5

Autosaving every 5 seconds


# 0. Importing story and template vectors and sums 

In [2]:
templates = pickle.load( open( 'c_template', "rb" ) )
recalls = pickle.load( open( 'c_recall', "rb" ) )
stories = pickle.load( open( 'c_stories', "rb" ) )

gran_stories = pickle.load( open( 'gran_stories', "rb" ) )
gran_templates = pickle.load( open( 'gran_templates', "rb" ) )
# sums
sums = pickle.load( open( "new_sums", "rb" ) )

In [71]:
top_ten = {}

# I. Finding the 3 Top Ten Words for each story
## A. Story Top Ten Words
### 1. Get the residuals of each participants

In [25]:
all_residuals = {}

for key in recalls:
    ### Get the residuals
    these_residuals = np.zeros((sums[key][0], 300))
    count = 0
    loc = key%10
    soc = round(key/10)*10
    # Make the inputs of the regression
    # location template
    l_temp = templates[loc].reshape(templates[loc].shape[0],-1)
    # social template
    s_temp = templates[soc].reshape(templates[soc].shape[0],-1)
    # Concatenating inputes
    inputs = np.concatenate((l_temp, s_temp), axis = 1)
    for i in range(0,3):
        n = recalls[key][i].shape[0]
        for j in range(0, n):
            output = np.copy(recalls[key][i][j])
            #output = output.reshape(output.shape[0],-1)
            model = LinearRegression().fit(inputs, output)
            ## Is subtracting the intercept correct?? 
            new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
            these_residuals[count, :] = new_vec
            count += 1
    all_residuals[key] = these_residuals

### 2. For each participant correlate their residuals with each word in the story

In [56]:
all_corrs = {}

for key in recalls:
    # turn key into a string
    str_key = str(key)
    # get the number of words
    n_words = gran_stories[str_key]['wvs'].shape[0]
    # get the number of participants
    n_participants = all_residuals[key].shape[0]
    # instantiate the vector n_words x n_participants
    corr_vals = np.zeros((n_words, n_participants))
    # Iterate through all words
    for word in range(0, n_words):
        # retrieve and reshape the vector for the word
        this_word = gran_stories[str_key]['wvs'][word]
        this_word = this_word.reshape(this_word.shape[0],-1)
        # Iterate through each participant
        for participant in range(0,n_participants):
            # Retrieve and reshape the vector for the participants recall
            this_partc = all_residuals[key][participant, :]
            this_partc = this_partc.reshape(this_partc.shape[0],-1)
            # Correlate the word and the participant recall
            this_correl = np.corrcoef(this_word.T, this_partc.T)
            # Store in corr_vals vector
            corr_vals[word][participant] = this_correl[0,1]
    # store corr_vals in dict
    all_corrs[key] = corr_vals

### 3. Avg (or would it be better to sum) across all participants

In [62]:
all_avgs = {}
all_sums = {}

for key in all_corrs:
    avg_vals = np.mean(all_corrs[key], axis = 1)
    sum_vals = np.sum(all_corrs[key], axis = 1)
    all_avgs[key] = avg_vals
    all_sums[key] = sum_vals

### 4. Index the 10 largest numbers

In [64]:
all_indices = {}

for key in all_avgs:
    print(key)
    this_corr = all_avgs[key]
    story_index = this_corr.argsort()[-10:][::-1]
    story_index_sums = all_sums[key].argsort()[-10:][::-1]
    print(story_index, story_index_sums)
    all_indices[key] = story_index

11
[ 46  21 116 635 656 544 577 135 103 440] [ 46  21 116 635 656 544 577 135 103 440]
33
[676 337 693 191  94 378 268 353 225 360] [676 337 693 191  94 378 268 353 225 360]
44
[399 500 471 497 259 279 236 263 254 392] [399 500 471 497 259 279 236 263 254 392]
23
[ 75  97 302 590 145 119 559 113 385 494] [ 75  97 302 590 145 119 559 113 385 494]
12
[430 114 176 484 360 376  39 155 280 460] [430 114 176 484 360 376  39 155 280 460]
21
[  0 446 181 390 252 272 532 436 488  29] [  0 446 181 390 252 272 532 436 488  29]
13
[403 437 126 361 539 337 180 423 103  35] [403 437 126 361 539 337 180 423 103  35]
42
[ 99 339 415   2 399 665 611 548 172 584] [ 99 339 415   2 399 665 611 548 172 584]
43
[460  49  19  28 433  74 159 355 423 339] [460  49  19  28 433  74 159 355 423 339]
32
[265 179 190  53 197 619 397 368 298 435] [265 179 190  53 197 619 397 368 298 435]
34
[ 91 129  32   2 514 235  65 477 354 231] [ 91 129  32   2 514 235  65 477 354 231]
22
[642  38 273 716 734 142 137  86  52  46

### 5. Get the Top Ten Words for the Story

In [73]:
for key in all_indices:
    print(key)
    str_key = str(key)
    this_index = all_indices[key]
    top_words = []
    for i in range(0, 10):
        top_words.append(gran_stories[str_key]['words'][this_index[i]])
    print(top_words)
    top_ten[key] = {'story': top_words}

11
['he', 'he', 'he', 'he', 'he', 'he', 'she', 'she', 'she', 'she']
33
['bianci', 'cheese', 'cheese', 'cheese', 'cheese', 'cheese', 'cheeses', 'cheeses', 'cheeses', 'cheeses']
44
['headshe', 'she', 'she', 'she', 'she', 'she', 'she', 'she', 'she', 'she']
23
['thanksgivin', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe']
12
['intolerability', 'they', 'they', 'they', 'they', 'they', 'they', 'they', 'they', 'they']
21
['alvira', 'alvira', 'alvira', 'alvira', 'alvira', 'alvira', 'alvira', 'alvira', 'alvira', 'alvira']
13
['emmanuella', 'emmanuella', 'emmanuella', 'emmanuella', 'emmanuella', 'lineliam', 'emmanuella', 'emmanuella', 'emmanuella', 'emmanuella']
42
['anna', 'anna', 'anna', 'anna', 'anna', 'anna', 'anna', 'anna', 'anna', 'anna']
43
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
32
['to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to']
34
['fund', 'alma', 'alma', 'alma', 'alma', 'alma', 'alma', 'alma', 'alma', '.']
22
['fived', 'had', 'ha

## B. Location Top Ten Words
### 1. Get the residuals of each participants

In [84]:
all_residuals = {}

for key in recalls:
    ### Get the residuals
    these_residuals = np.zeros((sums[key][0], 300))
    count = 0
    soc = round(key/10)*10
    # Make the inputs of the regression
    # story vector
    story_vec = stories[key].reshape(stories[key].shape[0],-1)
    # social template
    s_temp = templates[soc].reshape(templates[soc].shape[0],-1)
    # Concatenating inputes
    inputs = np.concatenate((story_vec, s_temp), axis = 1)
    for i in range(0,3):
        n = recalls[key][i].shape[0]
        for j in range(0, n):
            output = np.copy(recalls[key][i][j])
            #output = output.reshape(output.shape[0],-1)
            model = LinearRegression().fit(inputs, output)
            ## Is subtracting the intercept correct?? 
            new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
            these_residuals[count, :] = new_vec
            count += 1
    all_residuals[key] = these_residuals

### 2. For each participant correlate their residuals with each word in the location template

In [85]:
all_corrs = {}

for key in recalls:
    # get the loc schema
    loc = key%10
    # get the number of words
    n_words = gran_templates[loc]['wvs'].shape[0]
    # get the number of participants
    n_participants = all_residuals[key].shape[0]
    # instantiate the vector n_words x n_participants
    corr_vals = np.zeros((n_words, n_participants))
    # Iterate through all words
    for word in range(0, n_words):
        # retrieve and reshape the vector for the word
        this_word = gran_templates[loc]['wvs'][word]
        this_word = this_word.reshape(this_word.shape[0],-1)
        # Iterate through each participant
        for participant in range(0,n_participants):
            # Retrieve and reshape the vector for the participants recall
            this_partc = all_residuals[key][participant, :]
            this_partc = this_partc.reshape(this_partc.shape[0],-1)
            # Correlate the word and the participant recall
            this_correl = np.corrcoef(this_word.T, this_partc.T)
            # Store in corr_vals vector
            corr_vals[word][participant] = this_correl[0,1]
    # store corr_vals in dict
    all_corrs[key] = corr_vals

### 3. Avg (or would it be better to sum) across all participants

In [86]:
all_avgs = {}
all_sums = {}

for key in all_corrs:
    avg_vals = np.mean(all_corrs[key], axis = 1)
    sum_vals = np.sum(all_corrs[key], axis = 1)
    all_avgs[key] = avg_vals
    all_sums[key] = sum_vals

### 4. Index the 10 largest numbers

In [87]:
all_indices = {}

for key in all_avgs:
    print(key)
    this_corr = all_avgs[key]
    story_index = this_corr.argsort()[-10:][::-1]
    story_index_sums = all_sums[key].argsort()[-10:][::-1]
    print(story_index, story_index_sums)
    all_indices[key] = story_index

11
[14  0  6  1 26  4  2 22 12 24] [14  0  6  1 26  4  2 22 12 24]
33
[35 17 23  1  9  3  8 22  0  2] [35 17 23  1  9  3  8 22  0  2]
44
[24 51 61 37 45 58 30 13 16 19] [24 51 61 37 45 58 30 13 16 19]
23
[ 1 23  3  9  0  8  2 22 18 54] [ 1 23  3  9  0  8  2 22 18 54]
12
[27 11 53 16 47 45 19 32 29 59] [27 11 53 16 47 45 19 32 29 59]
21
[ 0  1 26  6 30 24 31 29 17 27] [ 0  1 26  6 30 24 31 29 17 27]
13
[35 32 43 10 34 54 18 45 29 60] [35 32 43 10 34 54 18 45 29 60]
42
[27 12 14 16 32 31  2 10  1  6] [27 12 14 16 32 31  2 10  1  6]
43
[54 18  8 22  0  2  3  9 23  1] [54 18  8 22  0  2  3  9 23  1]
32
[11 53 20 42 60 30 45  8 59 29] [11 53 20 42 60 30 45  8 59 29]
34
[ 0 49  7 26 42 47 64 14 34 48] [ 0 49  7 26 42 47 64 14 34 48]
22
[34 45 47 27 28 53 11 40 16 55] [34 45 47 27 28 53 11 40 16 55]
41
[27 15  4 17 29  0  1 26  6 25] [27 15  4 17 29  0  1 26  6 25]
14
[ 0 49  7 26 42 14 47  2 13 58] [ 0 49  7 26 42 14 47  2 13 58]
24
[ 0 49 26 42  7 55 14  2 33  5] [ 0 49 26 42  7 55 14  2 33

### 5. Get the Top Ten Words for the Location

In [90]:
for key in all_indices:
    # get the loc schema
    loc = key%10
    this_index = all_indices[key]
    top_words = []
    for i in range(0, 10):
        top_words.append(gran_templates[loc]['words'][this_index[i]])
    print(top_words)
    top_ten[key]['loc'] = top_words

['does', 'restaurant', 'restaurant', 'restaurant', 'restaurant', 'is', 'critic', 'like', 'like', 'food']
['buy', 'purchasing', 'store', 'store', 'store', 'store', 'grocery', 'grocery', 'grocery', 'grocery']
['is', 'is', 'is', 'is', 'is', 'class', 'class', 'class', 'class', 'class']
['store', 'store', 'store', 'store', 'grocery', 'grocery', 'grocery', 'grocery', 'groceries', 'groceries']
['they', 'to', 'and', 'when', 'are', 'which', 'arrive', 'departs', 'until', 'in']
['restaurant', 'restaurant', 'restaurant', 'restaurant', 'food', 'food', 'food', 'ordering', 'order', 'being']
['buy', 'pick', 'which', 'picking', 'to', 'groceries', 'groceries', 'do', 'do', 'do']
['they', 'boarding', 'boarding', 'when', 'departs', 'flight', 'customer', 'getting', 'airport', 'airport']
['groceries', 'groceries', 'grocery', 'grocery', 'grocery', 'grocery', 'store', 'store', 'store', 'store']
['to', 'and', 'at', 'at', 'on', 'their', 'which', 'through', 'in', 'until']
['lecture', 'lecture', 'lecture', 'lectur

## C. Social Top Ten Words
### 1. Get the residuals of each participants

In [93]:
all_residuals = {}

for key in recalls:
    ### Get the residuals
    these_residuals = np.zeros((sums[key][0], 300))
    count = 0
    soc = round(key/10)*10
    # Make the inputs of the regression
    # story vector
    story_vec = stories[key].reshape(stories[key].shape[0],-1)
    # location template
    l_temp = templates[loc].reshape(templates[loc].shape[0],-1)
    # Concatenating inputes
    inputs = np.concatenate((story_vec, l_temp), axis = 1)
    for i in range(0,3):
        n = recalls[key][i].shape[0]
        for j in range(0, n):
            output = np.copy(recalls[key][i][j])
            #output = output.reshape(output.shape[0],-1)
            model = LinearRegression().fit(inputs, output)
            ## Is subtracting the intercept correct?? 
            new_vec = output - np.dot(model.coef_, inputs.T)- model.intercept_
            these_residuals[count, :] = new_vec
            count += 1
    all_residuals[key] = these_residuals

### 2. For each participant correlate their residuals with each word in the location template

In [94]:
all_corrs = {}

for key in recalls:
    # get the soc schema
    soc = round(key/10)*10
    # get the number of words
    n_words = gran_templates[soc]['wvs'].shape[0]
    # get the number of participants
    n_participants = all_residuals[key].shape[0]
    # instantiate the vector n_words x n_participants
    corr_vals = np.zeros((n_words, n_participants))
    # Iterate through all words
    for word in range(0, n_words):
        # retrieve and reshape the vector for the word
        this_word = gran_templates[soc]['wvs'][word]
        this_word = this_word.reshape(this_word.shape[0],-1)
        # Iterate through each participant
        for participant in range(0,n_participants):
            # Retrieve and reshape the vector for the participants recall
            this_partc = all_residuals[key][participant, :]
            this_partc = this_partc.reshape(this_partc.shape[0],-1)
            # Correlate the word and the participant recall
            this_correl = np.corrcoef(this_word.T, this_partc.T)
            # Store in corr_vals vector
            corr_vals[word][participant] = this_correl[0,1]
    # store corr_vals in dict
    all_corrs[key] = corr_vals

In [95]:
### 3. Avg (or would it be better to sum) across all participants

In [96]:
all_avgs = {}
all_sums = {}

for key in all_corrs:
    avg_vals = np.mean(all_corrs[key], axis = 1)
    sum_vals = np.sum(all_corrs[key], axis = 1)
    all_avgs[key] = avg_vals
    all_sums[key] = sum_vals

In [97]:
### 4. Index the 10 largest numbers

In [98]:
all_indices = {}

for key in all_avgs:
    print(key)
    this_corr = all_avgs[key]
    story_index = this_corr.argsort()[-10:][::-1]
    story_index_sums = all_sums[key].argsort()[-10:][::-1]
    print(story_index, story_index_sums)
    all_indices[key] = story_index

11
[48 78 49 53 22 56 77 35 34 57] [48 78 49 53 22 56 77 35 34 57]
33
[53  1 14 44  2 18  0 49 29 26] [53  1 14 44  2 18  0 49 29 26]
44
[33 32 59 24 40 66 38 45 51 31] [33 32 59 24 40 66 38 45 51 31]
23
[48  0  1 22 20 29 42  9 30 13] [48  0  1 22 20 29 42  9 30 13]
12
[56 77 13  9 63 49 58 66 61 33] [56 77 13  9 63 49 58 66 61 33]
21
[20 44 48  0  1 10  5 18  3 21] [20 44 48  0  1 10  5 18  3 21]
13
[78 56 77 53 64 10 74 48 49 51] [78 56 77 53 64 10 74 48 49 51]
42
[33 59 50 70 58 45 38 68 51 31] [33 59 50 70 58 45 38 68 51 31]
43
[ 7 61 33 60 68 31 51 19 70 50] [ 7 61 33 60 68 31 51 19 70 50]
32
[44 53  1 14 31 52 48  6 37 13] [44 53  1 14 31 52 48  6 37 13]
34
[53 14  1 26 12 30 28 33  8 44] [53 14  1 26 12 30 28 33  8 44]
22
[40 32 18 44 21 26 10 13 30 43] [40 32 18 44 21 26 10 13 30 43]
41
[33 31 68 51 32 44 63 19 59 38] [33 31 68 51 32 44 63 19 59 38]
14
[49 35 48 78 87  5 14 18 27  0] [49 35 48 78 87  5 14 18 27  0]
24
[48 20  5 22  0  1 44  2 36 10] [48 20  5 22  0  1 44  2 36

In [99]:
### 5. Get the Top Ten Words for the Location

In [100]:
for key in all_indices:
    # get the loc schema
    soc = round(key/10)*10
    this_index = all_indices[key]
    top_words = []
    for i in range(0, 10):
        top_words.append(gran_templates[soc]['words'][this_index[i]])
    print(top_words)
    top_ten[key]['soc'] = top_words

['he', 'wants', 'she', 'does', 'has', 'who', 'who', 'her', 'his', 'is']
['deal', 'deal', 'deal', 'with', 'business', 'business', 'business', 'business', 'stake', 'money']
['she', 'he', 'meets', 'is', 'is', 'is', 'person', 'begins', 'when', 'when']
['fiancee', 'wedding', 'wedding', 'dating', 'couple', 'ring', 'ring', 'ring', 'and', 'and']
['who', 'who', 'to', 'to', 'to', 'she', 'being', 'and', 'with', 'with']
['couple', 'who', 'fiancee', 'wedding', 'wedding', 'proposing', 'proposal', 'has', 'preparing', 'been']
['wants', 'who', 'who', 'does', 'break', 'break', 'them', 'he', 'she', 'breaking']
['she', 'meets', 'couple', 'couple', 'couple', 'begins', 'person', 'when', 'when', 'when']
['for', 'for', 'she', 'and', 'when', 'when', 'when', 'later', 'couple', 'couple']
['with', 'deal', 'deal', 'deal', 'the', 'the', 'the', 'the', 'the', 'the']
['deal', 'deal', 'deal', 'money', 'concluding', 'in', 'at', 'offer', 'offer', 'with']
['is', 'is', 'has', 'who', 'been', 'are', 'proposing', 'and', 'and'

# II. Presenting Top Ten Words

In [106]:
for story in top_ten:
    print(story)
    for keys in top_ten[story]:
        print(keys)
        print(top_ten[story][keys])

11
story
['he', 'he', 'he', 'he', 'he', 'he', 'she', 'she', 'she', 'she']
loc
['does', 'restaurant', 'restaurant', 'restaurant', 'restaurant', 'is', 'critic', 'like', 'like', 'food']
soc
['he', 'wants', 'she', 'does', 'has', 'who', 'who', 'her', 'his', 'is']
33
story
['bianci', 'cheese', 'cheese', 'cheese', 'cheese', 'cheese', 'cheeses', 'cheeses', 'cheeses', 'cheeses']
loc
['buy', 'purchasing', 'store', 'store', 'store', 'store', 'grocery', 'grocery', 'grocery', 'grocery']
soc
['deal', 'deal', 'deal', 'with', 'business', 'business', 'business', 'business', 'stake', 'money']
44
story
['headshe', 'she', 'she', 'she', 'she', 'she', 'she', 'she', 'she', 'she']
loc
['is', 'is', 'is', 'is', 'is', 'class', 'class', 'class', 'class', 'class']
soc
['she', 'he', 'meets', 'is', 'is', 'is', 'person', 'begins', 'when', 'when']
23
story
['thanksgivin', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe', 'chloe']
loc
['store', 'store', 'store', 'store', 'grocery', 'grocery', 'gro